In [1]:
import os
import matplotlib.pyplot as plt

from pathlib import Path
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from utils import prepare_agreement_score_dataset, prepare_false_positive_dataset
import spikesorters as ss

In [45]:
os.environ["TMP"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["TMPDIR"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["TEMPDIR"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["ML_TEMPORARY_DIRECTORY"] = "/home/mclancy/SpikeConfidence/.tmp"

base_dir = Path('/home/mclancy/SpikeConfidence/')
analyses_dir = base_dir / 'analyses'
spike_sorter_dir = base_dir / 'spikesorters'

# # kilosort2
# !git clone https://github.com/MouseLand/Kilosort2.git
# kilosort2_path = spike_sorter_dir / 'Kilosort2.5'
# ss.Kilosort2Sorter.set_kilosort2_path(kilosort2_path)

ss.IronClustSorter.set_ironclust_path(spike_sorter_dir / 'ironclust')

dataset_path = Path("/home/mclancy/SpikeConfidence/analyses/recordings_50cells_SqMEA_2020/recordings_50cells_SqMEA-10-15_600.0_10.0uV_21-01-2020_18-12.h5")
# session = SpikeSorter(dataset_path)
# recording = session.get_recording()
# ground_truth = session.get_ground_truth()
#
#
# sorting = session.get_sorting(sorter_name='herdingspikes')

# sorter_names = ['klusta', 'tridesclous', 'mountainsort4',
#                 'ironclust', 'kilosort', 'kilosort2', 'kilosort2_5',
#                 'waveclus']

sorter_names = ['herdingspikes', 'spykingcircus', 'ironclust', 'tridesclous', 'mountainsort4']

metric_names = ["num_spikes", "firing_rate", "presence_ratio",
                "isi_violation", "amplitude_cutoff", "snr",
                "max_drift", "cumulative_drift", "silhouette_score",
                "isolation_distance", "l_ratio",
                "nn_hit_rate", "nn_miss_rate", "d_prime"]

Setting IRONCLUST_PATH environment variable for subprocess calls to: /home/mclancy/SpikeConfidence/spikesorters/ironclust


In [24]:
# Agreement score regression
X, y = prepare_agreement_score_dataset(
    dataset_path=dataset_path,
    sorter_names=sorter_names,
    metric_names=metric_names,
    hungarian=True
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

linear_glm = sm.GLM(y_train, X_train, family=sm.families.Binomial())
linear_result = linear_glm.fit()

y_test_preds = linear_glm.predict(X_test)
# plt.plot(y, linear_result.predict(X), 'o', alpha=0.2);

ValueError: shapes (310,13) and (153,13) not aligned: 13 (dim 1) != 153 (dim 0)

In [46]:
# False positive classification

X, y = prepare_false_positive_dataset(
    dataset_path=dataset_path,
    sorter_names=sorter_names,
    metric_names=metric_names,
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

scaler = StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)

model = LogisticRegression()
model.fit(X_train_transformed, y_train)

X_test_transformed = scaler.transform(X_test)
model.score(X_test_transformed, y_test)

ValueError: n_components cannot be larger than min(n_features, n_classes - 1).

In [30]:
y_test_preds = model.predict(X_test)